In [1]:
import logging
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import openai
from openai.embeddings_utils import get_embedding
import chromadb
from chromadb.config import Settings
from openai.embeddings_utils import get_embedding

In [2]:
credential = DefaultAzureCredential()
# Retrieve secrets from Key Vault
key_vault_name = "microhack-key-vault"
key_vault_uri = f"https://{key_vault_name}.vault.azure.net"
client = SecretClient(vault_url=key_vault_uri, credential=credential)
logging.info("Retreiving secrets from Azure Key Vault.")
fm_api_key = client.get_secret("FORM-RECOGNIZER-KEY").value
fm_endpoint = client.get_secret("FORM-RECOGNIZER-ENDPOINT").value

# openai
openai_api_key = client.get_secret("OPENAI-KEY").value
openai_endpoint = client.get_secret("OPENAI-ENDPOINT").value

# chroma
chroma_address = client.get_secret("CHROMA-DB-ADDRESS").value

In [3]:
openai.api_type = "azure"
openai.api_key = openai_api_key
openai.api_base = openai_endpoint
openai.api_version = "2022-12-01"

In [4]:
# Chroma Client
client = chromadb.Client(Settings(chroma_api_impl="rest",
                                  chroma_server_host=chroma_address,
                                  chroma_server_http_port="8000"))

In [5]:
# Ping Chroma
client.heartbeat()

1680788221823651651000

In [6]:
client.list_collections()

[Collection(name=microhack-collection)]

In [8]:
# get a collection
collection = client.get_collection("microhack-collection")

In [9]:
collection.count()

629

In [10]:
question = "What is statworx?"
query_embedding = get_embedding(question, engine="microhack-ada-text-embedding")

collection.query(
    query_embeddings=[query_embedding],
    n_results=2,
    include=["documents"]
)

{'ids': [['id261', 'id511']],
 'embeddings': None,
 'documents': [['STATWORX is a consulting and development company for Data Science, Machine Learning and Al with headquarters in Frankfurt am Main and Zurich. We support organizations end-2-end in the data-driven optimization of their products, services and processes. Thereby, we draw on a wealth of experience from more than 200 cross-industry and cross-functional Al strategy and development projects, from which you benefit in terms of best practices and experience.',
   'STATWORX is a consulting and development company for Data Science, Machine Learning and Al with headquarters in Frankfurt am Main and Zurich. We support organizations end-2-end in the data-driven optimization of their products, services and processes. Thereby, we draw on a wealth of experience from more than 200 cross-industry and cross-functional Al strategy and development projects, from which you benefit in terms of best practices and experience.']],
 'metadatas': 

In [ ]:
# Delete a Collection 
#client.delete_collection(<collection_name>)